In [2]:
import os
import pandas as pd
import numpy as np

OLD_PATH = r"C:\Users\Lenovo\Documents\github\seguimiento-de-noticias\data\raw\old"
archivos = [f for f in os.listdir(OLD_PATH) if f.endswith('.csv') and not f.startswith('~')]
CODIFICACIONES = ['utf-8', 'latin1', 'windows-1252']

MAPEO_ARCHIVOS = {
    'analisisdigital': {
        'medio': 'analisisdigital',
        'titulo': 'titulo',
        'contenido': ['contenido'],
        'enlace': 'enlace',
        'fecha': 'fecha',
        'copete': None,
        'descripcion': None,
        'id': 'id'
    },
    'apfdigital': {
        'medio': 'apfdigital',
        'titulo': 'titulo',
        'contenido': ['contenido'],
        'enlace': 'url',
        'fecha': 'fecha',
        'copete': None,
        'descripcion': None,
        'id': 'id'
    },
    'eldiario': {
        'medio': 'eldiario',
        'titulo': 'titulo',
        'contenido': ['descripcion'],  # 'contenido' NO está, usamos 'descripcion'
        'enlace': 'enlace',
        'fecha': 'fecha',
        'copete': None,
        'descripcion': 'descripcion',
        'id': 'id'
    },
    'elheraldo': {
        'medio': 'elheraldo',
        'titulo': 'titulo',
        'contenido': ['copete', 'contenido'],
        'enlace': 'enlace',
        'fecha': 'fecha',
        'copete': 'copete',
        'descripcion': None,
        'id': 'id'
    },
    'elonce': {
        'medio': 'elonce',
        'titulo': 'titulo',
        'contenido': ['contenido'],
        'enlace': 'url',
        'fecha': 'fecha',
        'copete': None,
        'descripcion': None,
        'id': 'id'
    },
    'unodigital': {
        'medio': 'unodigital',
        # Tomamos 'titulo_x' si está, si no 'titulo_y'
        'titulo': ['titulo_x', 'titulo_y'],
        'contenido': ['contenido'],
        'enlace': 'enlace',
        'fecha': 'fecha',
        'copete': None,
        'descripcion': None,
        'id': 'id'
    }
}

def cargar_csv_multi_encoding(path, codificaciones):
    for cod in codificaciones:
        try:
            return pd.read_csv(path, encoding=cod)
        except Exception:
            continue
    print(f"❌ No se pudo cargar {os.path.basename(path)} con ninguna codificación.")
    return None

def parse_fecha_flexible(fecha):
    if pd.isnull(fecha): return np.nan
    for fmt in ("%Y-%m-%d", "%Y-%m-%d %H:%M", "%d/%m/%Y"):
        try:
            return pd.to_datetime(fecha, format=fmt)
        except: continue
    return pd.to_datetime(fecha, errors='coerce')

dfs = []
for archivo in archivos:
    archivo_nombre = archivo.replace('.csv', '').lower()
    config = MAPEO_ARCHIVOS.get(archivo_nombre)
    if config is None:
        print(f"⚠️ Archivo {archivo} no está en el dict de mapeo, se saltea.")
        continue
    path = os.path.join(OLD_PATH, archivo)
    df = cargar_csv_multi_encoding(path, CODIFICACIONES)
    if df is not None:
        df = df.copy()
        df.columns = df.columns.str.lower().str.strip()

        # Titulo: buscar el primero que exista
        if isinstance(config['titulo'], list):
            for col in config['titulo']:
                if col in df.columns:
                    titulo = df[col]
                    break
            else:
                titulo = pd.NA
        else:
            titulo = df[config['titulo']] if config['titulo'] in df.columns else pd.NA

        # Contenido: concatenar todas las columnas que correspondan
        contenido_cols = [col for col in config['contenido'] if col and col in df.columns]
        contenido = df[contenido_cols].fillna('').astype(str).agg(' '.join, axis=1) if contenido_cols else pd.Series(['']*len(df))
        
        # Si hay copete o descripcion, agregamos (sólo si existen)
        if config.get('copete') and config['copete'] in df.columns:
            contenido = df[config['copete']].fillna('') + '. ' + contenido
        if config.get('descripcion') and config['descripcion'] in df.columns:
            contenido = df[config['descripcion']].fillna('') + '. ' + contenido

        # Enlace
        enlace = df[config['enlace']] if config['enlace'] in df.columns else pd.NA

        # Fecha
        fecha = df[config['fecha']].apply(parse_fecha_flexible) if config['fecha'] in df.columns else pd.NaT

        # Medio
        medio = config['medio']

        # ID (generar uno nuevo por seguridad)
        id_nuevo = [f"{medio}_{i}" for i in range(len(df))]

        # Construir df final de este archivo
        df_final = pd.DataFrame({
            'id_nuevo': id_nuevo,
            'medio': medio,
            'fecha': fecha,
            'titulo': titulo,
            'contenido': contenido,
            'enlace': enlace
        })

        dfs.append(df_final)
        print(f"✅ Procesado {archivo}: {df_final.shape}")

# Concatenar todo y deduplicar por enlace
noticias = pd.concat(dfs, ignore_index=True)
noticias = noticias.drop_duplicates(subset='enlace')

print(f"\n✅ Total noticias unificadas: {len(noticias)}")
print(noticias.head())


✅ Procesado analisisdigital.csv: (1561, 6)
✅ Procesado apfdigital.csv: (1076, 6)
✅ Procesado eldiario.csv: (130, 6)
✅ Procesado elheraldo.csv: (289, 6)
✅ Procesado elonce.csv: (1099, 6)
✅ Procesado unodigital.csv: (2486, 6)

✅ Total noticias unificadas: 6641
            id_nuevo            medio               fecha  \
0  analisisdigital_0  analisisdigital 2025-07-25 18:19:00   
1  analisisdigital_1  analisisdigital 2025-07-25 17:05:00   
2  analisisdigital_2  analisisdigital 2025-07-25 16:42:00   
3  analisisdigital_3  analisisdigital 2025-07-25 13:39:00   
4  analisisdigital_4  analisisdigital 2025-07-25 12:50:00   

                                              titulo  \
0  Frigerio analizó junto a su gabinete medidas p...   
1  El Gobierno presentó la renovada flota de cole...   
2  OSER firmó un convenio con el Colegio de Bioqu...   
3  “Lo acontecido con las horas TIC en Jóvenes y ...   
4  El 2 de agosto comenzará el preingreso en la F...   

                                     

In [3]:
# Clasificar palabras clave

import re
import pandas as pd
import numpy as np

# --- Diccionario con nombres y apellidos ---
intendente_info = [
    {"apellido": "Romero",    "nombres": ["Rosario"],        "full": "Rosario Romero"},
    {"apellido": "Azcué",     "nombres": ["Francisco"],      "full": "Francisco Azcué"},
    {"apellido": "Davico",    "nombres": ["Mauricio"],       "full": "Mauricio Davico"},
    {"apellido": "Lauritto",  "nombres": ["José", "Eduardo"],"full": "José Eduardo Lauritto"},
    {"apellido": "Bogdan",    "nombres": ["Dora"],           "full": "Dora Bogdan"},
    {"apellido": "Monjo",     "nombres": ["Claudia"],        "full": "Claudia Monjo"},
]

# --- Palabras a excluir antes (contexto de calle, avenida, barrio) ---
exclusiones_espaciales = ["calle", "avenida", "barrio"]

def es_mencion_valida(texto, apellido, nombres):
    """
    Evalúa si hay mención válida al intendente por apellido.
    """
    if pd.isnull(texto): return 0

    texto = str(texto)
    m = 0

    # 1. Chequear "intendente/a + apellido"
    if re.search(rf"(intendente|intendenta)\s+{apellido}", texto, re.IGNORECASE):
        m += 5  # peso alto

    # 2. Chequear nombre completo (cualquiera de los nombres + apellido)
    for nombre in nombres:
        if re.search(rf"{nombre}\s+{apellido}", texto, re.IGNORECASE):
            m += 3

    # 3. Chequear solo apellido (con reglas de exclusión)
    # Buscamos todas las posiciones del apellido en el texto
    for match in re.finditer(rf"\b{apellido}\b", texto, re.IGNORECASE):
        idx = match.start()
        # Obtener las palabras antes y después
        antes = texto[:idx].split()
        despues = texto[match.end():].split()
        palabra_antes = antes[-1] if antes else ''
        palabra_despues = despues[0] if despues else ''

        # a) Palabra anterior es exclusión espacial
        if palabra_antes.lower() in exclusiones_espaciales:
            continue
        # b) Palabra anterior o siguiente es mayúscula, ≥4 letras, y NO es nombre permitido
        if (len(palabra_antes) >= 4 and palabra_antes[0].isupper() and palabra_antes not in nombres):
            continue
        if (len(palabra_despues) >= 4 and palabra_despues[0].isupper() and palabra_despues not in nombres):
            continue
        # Si pasa los filtros, suma mención
        m += 1
    return m

def detectar_intendente(texto):
    # Aplica a cada intendente y devuelve el de mayor puntaje de mención válida
    menciones = {info['full']: es_mencion_valida(texto, info['apellido'], info['nombres']) for info in intendente_info}
    max_m = max(menciones.values())
    if max_m == 0:
        return np.nan
    # Si hay empate, devuelve todos separados por /
    mas_nom = [k for k, v in menciones.items() if v == max_m]
    return " / ".join(mas_nom)

# --- Aplicar al dataframe de noticias ---
noticias['intendenteMasNombrado'] = noticias['contenido'].apply(detectar_intendente)

import re

# Lista de localidades
localidades = [
    'Paraná', 'Concordia', 'Gualeguaychú', 'Concepción del Uruguay', 'Gualeguay', 'Villaguay'
]
exclusiones_espaciales = ["calle", "avenida", "barrio"]

def es_mencion_valida_localidad(texto, localidad):
    """
    Retorna True si la mención de localidad es válida (no está precedida por calle/avenida/barrio)
    """
    if pd.isnull(texto): return 0
    texto = str(texto)
    contador = 0
    for match in re.finditer(rf"\b{re.escape(localidad)}\b", texto, re.IGNORECASE):
        idx = match.start()
        antes = texto[:idx].split()
        palabra_antes = antes[-1] if antes else ''
        # Excluir si está precedida por "calle", "avenida" o "barrio"
        if palabra_antes.lower() in exclusiones_espaciales:
            continue
        contador += 1
    return contador

def detectar_localidad(texto):
    menciones = {loc: es_mencion_valida_localidad(texto, loc) for loc in localidades}
    max_m = max(menciones.values())
    if max_m == 0:
        return np.nan
    mas_nom = [k for k, v in menciones.items() if v == max_m]
    return " / ".join(mas_nom)

# --- Aplicar al DataFrame ---
noticias['localidadMasNombrada'] = noticias['contenido'].apply(detectar_localidad)

# -- Cantidad de noticias con localidad (ciudad) etiquetada --
if 'localidadMasNombrada' in noticias.columns:
    total_ciudad = noticias['localidadMasNombrada'].notna().sum()
    print(f"🏙️ Noticias con localidad (ciudad) etiquetada: {total_ciudad}")
    print("🔎 Ranking de noticias por ciudad más nombrada:")
    print(noticias['localidadMasNombrada'].value_counts(dropna=True))
else:
    print("⚠️ No existe columna 'localidadMasNombrada'.")

# -- Cantidad de noticias con intendente etiquetado --
if 'intendenteMasNombrado' in noticias.columns:
    total_intendente = noticias['intendenteMasNombrado'].notna().sum()
    print(f"\n🧑‍💼 Noticias con intendente etiquetado: {total_intendente}")
    print("🔎 Ranking de noticias por intendente más nombrado:")
    print(noticias['intendenteMasNombrado'].value_counts(dropna=True))
else:
    print("⚠️ No existe columna 'intendenteMasNombrado'.")



🏙️ Noticias con localidad (ciudad) etiquetada: 1828
🔎 Ranking de noticias por ciudad más nombrada:
localidadMasNombrada
Paraná                                                                                1128
Concordia                                                                              231
Gualeguaychú                                                                           109
Concepción del Uruguay                                                                  81
Villaguay                                                                               64
Paraná / Concordia                                                                      33
Gualeguay                                                                               28
Paraná / Gualeguaychú                                                                   18
Paraná / Villaguay                                                                      18
Paraná / Concepción del Uruguay                              

In [4]:
import pandas as pd
import re

def segmentar_oraciones(texto):
    if pd.isnull(texto): return []
    # Separa por punto, signo de exclamación o interrogación
    return [s.strip() for s in re.split(r'(?<=[.!?])\s+', str(texto)) if s.strip()]

menciones = []
for idx, row in noticias.iterrows():
    oraciones = segmentar_oraciones(row['contenido'])
    for oracion in oraciones:
        menciones.append({
            'id_noticia': row['id_nuevo'] if 'id_nuevo' in row else row['id'],
            'medio': row['medio'],
            'fecha': row['fecha'],
            'titulo': row['titulo'],
            'oracion': oracion,
            'localidadMasNombrada': row.get('localidadMasNombrada', np.nan),
            'intendenteMasNombrado': row.get('intendenteMasNombrado', np.nan),
        })

df_menciones = pd.DataFrame(menciones)
print(f"📝 Total de menciones (oraciones): {len(df_menciones)}")
print(df_menciones.head(10))


📝 Total de menciones (oraciones): 85836
          id_noticia            medio               fecha  \
0  analisisdigital_0  analisisdigital 2025-07-25 18:19:00   
1  analisisdigital_0  analisisdigital 2025-07-25 18:19:00   
2  analisisdigital_0  analisisdigital 2025-07-25 18:19:00   
3  analisisdigital_0  analisisdigital 2025-07-25 18:19:00   
4  analisisdigital_0  analisisdigital 2025-07-25 18:19:00   
5  analisisdigital_0  analisisdigital 2025-07-25 18:19:00   
6  analisisdigital_0  analisisdigital 2025-07-25 18:19:00   
7  analisisdigital_0  analisisdigital 2025-07-25 18:19:00   
8  analisisdigital_0  analisisdigital 2025-07-25 18:19:00   
9  analisisdigital_0  analisisdigital 2025-07-25 18:19:00   

                                              titulo  \
0  Frigerio analizó junto a su gabinete medidas p...   
1  Frigerio analizó junto a su gabinete medidas p...   
2  Frigerio analizó junto a su gabinete medidas p...   
3  Frigerio analizó junto a su gabinete medidas p...   
4  Frige

In [5]:
import pysentimiento
import pandas as pd

# --- 1. Filtrar solo con intendente etiquetado ---
noticias_intendente = noticias[noticias['intendenteMasNombrado'].notna()].copy()
print(f"🔎 Noticias a clasificar (con intendente): {len(noticias_intendente)}")

menciones_intendente = df_menciones[df_menciones['intendenteMasNombrado'].notna()].copy()
print(f"🔎 Menciones a clasificar (con intendente): {len(menciones_intendente)}")

# --- 2. Análisis de sentimiento ---
analyzer = pysentimiento.create_analyzer(task="sentiment", lang="es")

def sentimiento_noticia(texto):
    resultado = analyzer.predict(str(texto))
    return pd.Series({
        'sentimiento_noticia': resultado.output,
        'proba_sentimiento_noticia': float(resultado.probas[resultado.output])
    })

def sentimiento_mencion(texto):
    resultado = analyzer.predict(str(texto))
    return pd.Series({
        'sentimiento_mencion': resultado.output,
        'proba_sentimiento_mencion': float(resultado.probas[resultado.output])
    })

# Aplicar en bloque
noticias_intendente[['sentimiento_noticia', 'proba_sentimiento_noticia']] = (
    noticias_intendente['contenido'].apply(sentimiento_noticia)
)
menciones_intendente[['sentimiento_mencion', 'proba_sentimiento_mencion']] = (
    menciones_intendente['oracion'].apply(sentimiento_mencion)
)

# --- 3. Tablas resumen ---
print("\n--- Resumen de sentimiento por noticia ---")
resumen_noticias = (
    noticias_intendente
    .groupby(['intendenteMasNombrado', 'sentimiento_noticia'])
    .size()
    .unstack(fill_value=0)
    .sort_index()
)
print(resumen_noticias)

print("\n--- Resumen de sentimiento por mención (oración) ---")
resumen_menciones = (
    menciones_intendente
    .groupby(['intendenteMasNombrado', 'sentimiento_mencion'])
    .size()
    .unstack(fill_value=0)
    .sort_index()
)
print(resumen_menciones)


import os


OUTPUT_PATH = r"C:/Users/Lenovo/Documents/github/seguimiento-de-noticias/data/processed"
os.makedirs(OUTPUT_PATH, exist_ok=True)

# 1. Noticias limpias
noticias_etiquetadas = noticias.copy()
noticias_etiquetadas.to_csv(os.path.join(OUTPUT_PATH, "noticias_etiquetadas.csv"), index=False)
print("✅ Exportado: noticias_etiquetadas.csv")
    
# 2. Menciones (oraciones)
df_menciones.to_csv(os.path.join(OUTPUT_PATH, "menciones_etiquetadas.csv"), index=False)
print("✅ Exportado: menciones_etiquetadas.csv")

# 3. Resumen por noticia
resumen_noticias.to_csv(os.path.join(OUTPUT_PATH, "resumen_noticias.csv"))
print("✅ Exportado: resumen_noticias.csv")

# 4. Resumen por mención
resumen_menciones.to_csv(os.path.join(OUTPUT_PATH, "resumen_menciones.csv"))
print("✅ Exportado: resumen_menciones.csv")

# 5. Muestra aleatoria de 100 menciones para control manual
muestra = df_menciones.sample(n=100, random_state=42) if len(df_menciones) >= 100 else df_menciones
muestra.to_csv(os.path.join(OUTPUT_PATH, "muestra_100_menciones.csv"), index=False)
print("✅ Exportado: muestra_100_menciones.csv")

# 6. Noticias y menciones con etiquetas 
noticias_intendente.to_csv(os.path.join(OUTPUT_PATH, "noticias_intendente_sentimiento.csv"), index=False)
menciones_intendente.to_csv(os.path.join(OUTPUT_PATH, "menciones_intendente_sentimiento.csv"), index=False)


C:\Users\Lenovo\AppData\Roaming\Python\Python313\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🔎 Noticias a clasificar (con intendente): 583
🔎 Menciones a clasificar (con intendente): 11610

--- Resumen de sentimiento por noticia ---
sentimiento_noticia                     NEG  NEU  POS
intendenteMasNombrado                                
Claudia Monjo                             1    4    0
Dora Bogdan                               0    2    3
Francisco Azcué                          28   30   11
José Eduardo Lauritto                     8    6    3
Mauricio Davico                           7   10    8
Rosario Romero                           31  170  249
Rosario Romero / José Eduardo Lauritto    1    9    2

--- Resumen de sentimiento por mención (oración) ---
sentimiento_mencion                      NEG   NEU   POS
intendenteMasNombrado                                   
Claudia Monjo                             14    65    13
Dora Bogdan                               11    43    26
Francisco Azcué                          394   636   280
José Eduardo Lauritto               

In [ ]:
noticias_intendente
menciones_intendente

Index(['id_noticia', 'medio', 'fecha', 'titulo', 'oracion',
       'localidadMasNombrada', 'intendenteMasNombrado', 'sentimiento_mencion',
       'proba_sentimiento_mencion'],
      dtype='object')